In [1]:
import pandas as pd
import requests
import io
import sqlite3

start_period = '2024-03'
conn = sqlite3.connect('/Users/Mark Rozenberg/Downloads/Macro-Indicators/macro_indicators_v2.db')

In [2]:
# data links
wage_url = 'https://sdmx.oecd.org/public/rest/data/OECD.SDD.TPS,DSD_EAR@DF_HOU_EAR,1.0/all?startPeriod=' + start_period + '&dimensionAtObservation=AllDimensions&format=csvfilewithlabels'
macro_ind_url = 'https://sdmx.oecd.org/public/rest/data/OECD.SDD.STES,DSD_STES_REVISIONS@DF_STES_REVISIONS,4.0/all?startPeriod=2000-01&dimensionAtObservation=AllDimensions&format=csvfilewithlabels'
unemp_url = 'https://sdmx.oecd.org/public/rest/data/OECD.SDD.TPS,DSD_LFS@DF_IALFS_UNE_M,1.0/all?startPeriod=2000-01&dimensionAtObservation=AllDimensions&format=csvfilewithlabels'
finance_url = 'https://sdmx.oecd.org/public/rest/data/OECD.SDD.STES,DSD_STES@DF_FINMARK,4.0/all?startPeriod=2000-01&dimensionAtObservation=AllDimensions&format=csvfilewithlabels'
cpi_url = 'https://sdmx.oecd.org/public/rest/data/OECD.SDD.TPS,DSD_PRICES@DF_PRICES_ALL,1.0/all?startPeriod=2024-01&dimensionAtObservation=AllDimensions&format=csvfilewithlabels'
cci_url = 'https://sdmx.oecd.org/public/rest/data/OECD.SDD.STES,DSD_STES@DF_CS,4.0/all?startPeriod=' + start_period + '&dimensionAtObservation=AllDimensions&format=csvfilewithlabels'

### CCI

In [12]:
cci_data = pd.read_csv(io.StringIO(requests.get(cci_url).content.decode('utf-8')), header=0)
cci_data.rename(columns={
'MEASURE': 'measure_code',
'Measure': 'measure_name',
'ADJUSTMENT': 'adjustment_code',
'Adjustment': 'adjustment_name',
'TRANSFORMATION': 'transformation_code',
'Transformation': 'transformation_name',
'TIME_PERIOD': 'time_period_code',
'Time_period': 'time_period_name',
'DECIMALS': 'decimals_code',
'Decimals': 'decimals_name'
}, inplace=True)
cci_data.columns = cci_data.columns.str.replace(' ', '_')
cci_data = cci_data[(cci_data['FREQ'] == 'M') & (cci_data['measure_code'] == 'CCICP')][['time_period_code', 'REF_AREA', 'OBS_VALUE']]


In [13]:
cci_data.head()

,time_period_code,REF_AREA,OBS_VALUE
0,2024-06,ITA,-17.10000
1,2024-07,ITA,-15.00000
5,2024-04,USA,83.06285
6,2024-05,USA,74.34771
481,2024-03,GBR,-15.25000


In [ ]:
# Insert new data into irlt table
for index, row in cci_data.iterrows():
    conn.execute('''
    INSERT INTO cci (time_period_code, REF_AREA, OBS_VALUE)
    VALUES (?, ?, ?)
    ON CONFLICT(time_period_code, REF_AREA) DO UPDATE SET
    OBS_VALUE = excluded.OBS_VALUE
    ''', (row['time_period_code'], row['REF_AREA'], row['OBS_VALUE']))

# Commit the changes
conn.commit()

OperationalError: ON CONFLICT clause does not match any PRIMARY KEY or UNIQUE constraint

In [8]:
pd.read_sql_query('''
SELECT count(*) FROM cci
where freq = 'M'
and measure_code = 'CCICP'
''', conn)

,count(*)
0,189


In [9]:
pd.read_sql_query('''
with part1 as(
SELECT distinct time_period_code, REF_AREA
FROM cci
where freq = 'M'
and measure_code = 'CCICP'
)
select count(*) from part1
''', conn)

,count(*)
0,189
